# Create occupation-level plots for ILO Scenario
Felix Zaussinger | 20.08.2021

## Core Analysis Goal(s)
1. Create stacked barplots similar to ILO 2019, page 132
2.
3.

## Key Insight(s)
1.
2.
3.

In [80]:
import os
import sys
import logging
from pathlib import Path

import numpy as np
import scipy as sp
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("paper")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("ticks")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define directory structure

In [3]:
# project directory
abspath = os.path.abspath('')
project_dir = str(Path(abspath).parents[0])

# sub-directories
data_raw = os.path.join(project_dir, "data", "raw")
data_interim = os.path.join(project_dir, "data", "interim")
data_processed = os.path.join(project_dir, "data", "processed")
figure_dir = os.path.join(project_dir, "reports", "figures")

Code ...

In [44]:
fpath = os.path.join(data_raw, "ilo_scenarios", "ilo_empl_scenarios_2030.xlsx")

dict_of_dfs = pd.read_excel(
    fpath,
    sheet_name=["energy_sustainability", "circular_economy"]
)

In [73]:
# rename dicts
title_dict = {
    "energy_sustainability": "Energy Sustainability Scenario",
    "circular_economy": "Circular Economy Scenario"
}

col_names = {
    "new_jobs_net": "New jobs",
    "new_jobs_absorbing": "New jobs absorbing laid-off workers",
    "jobs_lost_reallocated": "Jobs destroyed, reallocatable",
    "jobs_lost_net": "Jobs destroyed, not reallocatable"
}

#### Create stacked barplots similar to ILO 2019, page 132

In [ ]:
# ILO ES 2030
for scenario, df in dict_of_dfs.items():

    df["label"] = df["isco_code"].astype(str) + " - " + df["isco_occupation"]
    df = df.set_index("label")
    df = df.sort_values("net_change", ascending=True)
    df = df.drop(columns=["net_change", "isco_code", "isco_occupation", "skill_level"])
    df = df.rename(columns=col_names)

    # scale to millions
    df *= 1000

    # plot
    colors = {
        "New jobs": "#2c7bb6",
        "New jobs absorbing laid-off workers": "#abd9e9",
        "Jobs destroyed, reallocatable": "#fdae61",
        "Jobs destroyed, not reallocatable": "#d7191c"
    }

    ax = df.plot.barh(stacked=True, color=colors)
    ax.set_xlabel("Change in labour demand")
    ax.set_ylabel(None)
    ax.set_title(title_dict[scenario])
    ax.grid(axis='x')

    ax.xaxis.set_major_formatter(plt.FormatStrFormatter('%2d'))

    sns.despine()
    plt.legend(loc="lower right")
    plt.tight_layout()

    plt.savefig(
        os.path.join(figure_dir, "RQ2", "{}.png".format(scenario)),
        dpi=150,
        bbox_inches="tight"
    )

#### Create plots regarding skill level

In [115]:
tips = sns.load_dataset("tips")
tips
#dict_of_dfs["energy_sustainability"][["skill_level", "net_change"]].groupby("skill_level").sum()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [117]:
ce = dict_of_dfs["circular_economy"]
ce["positive"] = ce.new_jobs_absorbing + ce.new_jobs_net
ce["negative"] = ce.jobs_lost_reallocated + ce.jobs_lost_net
res = ce[["skill_level", "positive", "negative"]].groupby("skill_level").sum()
res *= 1000
res
ce

,isco_code,isco_occupation,skill_level,new_jobs_absorbing,new_jobs_net,jobs_lost_net,jobs_lost_reallocated,net_change,label,positive,negative
0,11,"Chief executives, senior officials and legisla...",H,244.0,375.0,-43.0,-244.0,332.0,"11 - Chief executives, senior officials and le...",619.0,-287.0
1,12,Administrative and commercial managers,H,596.0,797.0,-32.0,-596.0,765.0,12 - Administrative and commercial managers,1393.0,-628.0
2,13,Production and specialized services managers,H,1095.0,143.0,-1110.0,-1095.0,-967.0,13 - Production and specialized services managers,1238.0,-2205.0
3,14,"Hospitality, retail and other services managers",H,121.0,1035.0,-4.0,-121.0,1031.0,"14 - Hospitality, retail and other services ma...",1156.0,-125.0
4,19,Other managers,H,0.5,1.0,-1.0,-0.5,0.0,19 - Other managers,1.5,-1.5
5,21,Science and engineering professionals,H,1379.0,1274.0,-397.0,-1379.0,877.0,21 - Science and engineering professionals,2653.0,-1776.0
6,22,Health professionals,H,222.0,173.0,-77.0,-222.0,96.0,22 - Health professionals,395.0,-299.0
7,23,Teaching professionals,H,125.0,166.0,-58.0,-125.0,108.0,23 - Teaching professionals,291.0,-183.0
8,24,Business and administration professionals,H,823.0,837.0,-53.0,-823.0,784.0,24 - Business and administration professionals,1660.0,-876.0
9,25,Information and communications technology prof...,H,187.0,296.0,-31.0,-187.0,265.0,25 - Information and communications technology...,483.0,-218.0
